<a href="https://colab.research.google.com/github/brook-miller/2023mbai417/blob/main/3-class/california-housing-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Standard imports - used in most EDA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go

from datetime import datetime, timedelta
from dateutil.parser import parse
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [ ]:
#@title Loading California housing dataset
#@markdown description here: https://developers.google.com/machine-learning/crash-course/california-housing-data-description
from sklearn.datasets import fetch_california_housing

df = fetch_california_housing(as_frame=True).frame
df

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


In [ ]:
#@title Define, consistent train test split function
from sklearn.model_selection import train_test_split

def get_train_test(df):
  return train_test_split(df.loc[:, df.columns != "MedHouseVal"], df["MedHouseVal"], test_size = .3, random_state=77)

X_train, X_test, y_train, y_test = get_train_test(df)

# Evaluating Linear, XGBoost and MLP Regression Models

In [ ]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train, y_train)
l1 = linreg.score(X_train, y_train)
l2 = linreg.score(X_test, y_test)
print(f"R² of Linear Regression on training set: {l1:.3f}")
print(f"R² of Linear Regression on test set: {l2:.3f}")

R² of Linear Regression on training set: 0.609
R² of Linear Regression on test set: 0.598


In [ ]:
from xgboost import XGBRegressor, plot_importance

xgbmodel = XGBRegressor(n_estimators=100, max_depth=3, eta=0.3, subsample=.3, colsample_bytree=0.3, objective ='reg:squarederror')

xgbmodel.fit(X_train, y_train)
xg1 = xgbmodel.score(X_train, y_train)
xg2 = xgbmodel.score(X_test, y_test)
print(f"R² of XGBRegressor on training set: {xg1:.3f}")
print(f"R² of XGBRegressor on test set: {xg2:.3f}")

R² of XGBRegressor on training set: 0.761
R² of XGBRegressor on test set: 0.731


In [ ]:
from sklearn.neural_network import MLPRegressor
mlp = MLPRegressor(random_state=42)
mlp.fit(X_train, y_train)
m1 = mlp.score(X_train, y_train)
m2 = mlp.score(X_test, y_test)

print(f"R² of MLPRegressor on training set: {m1:.3f}")
print(f"R² of MLPRegressor on test set: {m2:.3f}")

R² of MLPRegressor on training set: 0.559
R² of MLPRegressor on test set: 0.533


In [ ]:
#@title building an "ensemble" scoring method to evaluate xgboost, mlpregressor and linear regression R^2 
#@markdown function returns a DataFrame so that we can see how performance changes as we make changes to the data
def ensemble_scores(df_, exp_name):
  X_train, X_test, y_train, y_test = get_train_test(df_)
  mlp = MLPRegressor(random_state=42)
  mlp.fit(X_train, y_train)
  m1 = mlp.score(X_train, y_train)
  m2 = mlp.score(X_test, y_test)
  model = XGBRegressor(n_estimators=100, max_depth=3, eta=0.3, subsample=.3, colsample_bytree=0.3, objective ='reg:squarederror')
  model.fit(X_train, y_train)
  xg1 = model.score(X_train, y_train)
  xg2 = model.score(X_test, y_test)
  linreg = LinearRegression()
  linreg.fit(X_train, y_train)
  l1 = linreg.score(X_train, y_train)
  l2 = linreg.score(X_test, y_test)
  df = pd.DataFrame([[exp_name, m1, m2, xg1, xg2, l1, l2]], columns=['ExpName', 'MLPTrain', 'MLPTest', 'XGBoostTrain', 'XGBTest','LinearTrain', 'LinearTest'])
  return df

In [ ]:
experiment_df = ensemble_scores(df, "baseline")
experiment_df

,ExpName,MLPTrain,MLPTest,XGBoostTrain,XGBTest,LinearTrain,LinearTest
0,baseline,0.559176,0.533395,0.761304,0.731158,0.608929,0.598024


In [ ]:
#@title using the sklearn standard scaler
from sklearn.preprocessing import StandardScaler
df_standard_scaled_features = df.copy()

col_names = df_standard_scaled_features.columns[:-1]
features = df_standard_scaled_features[col_names]

scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)

df_standard_scaled_features[col_names] = features

experiment_df = pd.concat([experiment_df, ensemble_scores(df_standard_scaled_features, "standardscaled")])
experiment_df

,ExpName,MLPTrain,MLPTest,XGBoostTrain,XGBTest,LinearTrain,LinearTest
0,baseline,0.559176,0.533395,0.761304,0.731158,0.608929,0.598024
0,standardscaled,0.794709,0.770966,0.761314,0.731159,0.608929,0.598024


In [ ]:
#@title recall there are some lognormal variables
fig = px.histogram(df, x="MedInc")
fig.show()

In [ ]:
from sklearn.preprocessing import power_transform

In [ ]:
df_temp = df.copy()

df_temp["MedInc"] = power_transform(df_temp[["MedInc"]])
df_temp["MedHouseVal"] = power_transform(df_temp[["MedHouseVal"]])

col_names = df_temp.columns[:-1]
features = df_temp[col_names]

scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)

df_temp[col_names] = features

experiment_df = pd.concat([experiment_df, ensemble_scores(df_temp, "+power on MedInc,MedHouseVal")])
experiment_df

,ExpName,MLPTrain,MLPTest,XGBoostTrain,XGBTest,LinearTrain,LinearTest
0,baseline,0.559176,0.533395,0.761304,0.731158,0.608929,0.598024
0,standardscaled,0.794709,0.770966,0.761314,0.731159,0.608929,0.598024
0,"+power on MedInc,MedHouseVal",0.757547,0.797845,0.775169,0.744179,0.634871,0.620415


In [ ]:
fig = px.histogram(df_temp, x="MedInc")
fig.show()

In [ ]:
def standard_power_transform(df_):
  df_temp = df_.copy()

  df_temp["MedInc"] = power_transform(df_temp[["MedInc"]])
  df_temp["MedHouseVal"] = power_transform(df_temp[["MedHouseVal"]])
  
  col_names = df_temp.columns[:-1]
  features = df_temp[col_names]

  scaler = StandardScaler().fit(features.values)
  features = scaler.transform(features.values)

  df_temp[col_names] = features
  return df_temp

## Evaluating the errors in linear regression model

In [ ]:
df_pred = standard_power_transform(df)

X_train, X_test, y_train, y_test = get_train_test(df_pred)
linreg = LinearRegression()
linreg.fit(X_train, y_train)

df_pred["pred"] = linreg.predict(df_pred.loc[:, df.columns != "MedHouseVal"])

df_pred

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal,pred
0,1.903289,0.982143,0.628559,-0.153758,-0.974429,-0.049597,1.052548,-1.327835,1.715878,1.403335
1,1.897081,-0.607019,0.327041,-0.263336,0.861439,-0.092512,1.043185,-1.322844,1.281389,1.308803
2,1.604361,1.856182,1.155620,-0.049016,-0.820777,-0.025843,1.038503,-1.332827,1.247506,1.210326
3,1.051321,1.856182,0.156966,-0.049833,-0.766028,-0.050329,1.038503,-1.337818,1.188835,1.033500
4,0.205878,1.856182,0.344711,-0.032906,-0.759847,-0.085616,1.038503,-1.337818,1.193798,0.434572
...,...,...,...,...,...,...,...,...,...,...
20635,-1.675311,-0.289187,-0.155023,0.077354,-0.512592,-0.049110,1.801647,-0.758826,-1.414289,-2.172605
20636,-0.676386,-0.845393,0.276881,0.462365,-0.944405,0.005021,1.806329,-0.818722,-1.433165,-1.491464
20637,-1.509426,-0.924851,-0.090318,0.049414,-0.369537,-0.071735,1.778237,-0.823713,-1.160803,-2.045023
20638,-1.323914,-0.845393,-0.040211,0.158778,-0.604429,-0.091225,1.778237,-0.873626,-1.293200,-1.852164


In [ ]:
#@title Comparing Linear Regression predicted to MedHouseVal
#@markdown values are power transformed, standard scaled.
fig = px.scatter(df_pred, x="MedHouseVal", y="pred", height=600, width=600)
fig.show()

In [ ]:
#@title Examining how the errors are distributed
df_pred["error"] = df_pred["pred"] - df_pred["MedHouseVal"]
fig = px.scatter(df_pred, x="Longitude", y="Latitude", color="error", height=600, width=600)
fig.show()

Looking at the distribution of colors on the graph, it's clear that the errors are correlated to location.  We see the dots clustering together by their colors.  It's obvious that Latitude and Longitude should not have a linear relationship to HouseMedVal.  We will look at clustering through KNeighbors regressor to define a better model.

# KNeighborsRegressor

In [ ]:
#@title evaluating performance of KNeighbors regressor
from sklearn.neighbors import KNeighborsRegressor

df_temp = df[["MedInc", "Latitude", "Longitude", "MedHouseVal"]]

X_train, X_test, y_train, y_test = get_train_test(df_temp)

knn = KNeighborsRegressor(n_neighbors=10)
knn.fit(X_train, y_train)

knn1 = knn.score(X_train, y_train)
knn2 = knn.score(X_test, y_test)
print(f"R² of KNN on training set: {knn1:.3f}")
print(f"R² of KNN on test set: {knn2:.3f}")

R² of KNN on training set: 0.812
R² of KNN on test set: 0.751


In [ ]:
#@title KNeighbors algorithm is sensitive to the number of neighbors
#@markdown we can iterate over a range of neighbors and then look at the performance of k neighbors (focus on the test set)
df_temp = df[["MedInc", "Latitude", "Longitude", "MedHouseVal"]]

X_train, X_test, y_train, y_test = get_train_test(df_temp)
rows =[]
neighbors_settings = range(2, 50)
for n_neighbors in neighbors_settings:
  knn = KNeighborsRegressor(n_neighbors=n_neighbors)
  knn.fit(X_train, y_train)
  knn1 = knn.score(X_train, y_train)
  knn2 = knn.score(X_test, y_test)
  rows.append({"neighbors" : n_neighbors, "val":knn1, "metric": "TrainR2" })
  rows.append({"neighbors" : n_neighbors, "val":knn2, "metric": "TestR2" })

knneval_df = pd.DataFrame.from_dict(rows, orient="columns")

fig = px.line(knneval_df, x="neighbors", y="val", color="metric")
fig.show()

In [ ]:
#@title KNeighbors also is sensitive to the scale of the features
#@markdown Use different scales and ranges until there is a clear peak on test set performance

df_temp = df[["MedInc", "Latitude", "Longitude", "MedHouseVal"]]

rows =[]
income_scale_range = range(1, 100)

for income_scale in income_scale_range:
  #iterating over scaling the MedInc from 1/1000 = 0.1% through 10%
  df_temp = df_temp.assign(MedInc = df["MedInc"] * income_scale / 1000)
  X_train, X_test, y_train, y_test = get_train_test(df_temp)
  knn = KNeighborsRegressor(n_neighbors=10)
  knn.fit(X_train, y_train)
  knn1 = knn.score(X_train, y_train)
  knn2 = knn.score(X_test, y_test)
  rows.append({"scale" : income_scale, "val":knn1, "metric": "TrainR2" })
  rows.append({"scale" : income_scale, "val":knn2, "metric": "TestR2" })

knneval_df = pd.DataFrame.from_dict(rows, orient="columns")

fig = px.line(knneval_df, x="scale", y="val", color="metric")
fig.show()

Discussion: KNeighbors performs well but....

KNeighbors does not generalize.  If we were trying to pick reasonable MedHouseValues in Nebraska (without more training data on Nebraska) this model will perform very poorly.  This lesson is useful as we generalize to more complex feature spaces.  If we have training data that finds all the "corners" of our highly dimensional feature spaces, then kmeans can perform very well.
  
 
With our increased abilities to augment and generate training data and transformer models to deliver embeddings, many complex ML inference tasks can become KNN problems.

TO DO NEXT: 
* Show how KNN Performs when only trained on the San Diego area
* Evaluate sensitivity of models to the percentage allocated to test
* Evaluate an ensemble of KNN with the other models to improve performance without overfitting 

---
### It is useful to get feature importances from Linear Regression and XGBoost models periodically in order to determine which features improve the model.


In [ ]:
importance_df = pd.DataFrame(data={
    'attribute': X_train.columns,
    'importance': linreg.coef_
})
importances = importance_df.sort_values(by='importance', ascending=False)

fig = px.bar(importances, x='attribute', y='importance')
fig.show()

In [ ]:
importance_df = pd.DataFrame(data={
    'attribute': X_train.columns,
    'importance': xgbmodel.feature_importances_
})
importances = importance_df.sort_values(by='importance', ascending=False)

fig = px.bar(importances, x='attribute', y='importance')
fig.show()

#Code review
* DRY - don't repeat yourself.  Lots of copied code. Some of this repetition makes the notebook a bit easier to follow since we are focused on both the code and the output for this class.
* Variables declared local in functions have same names as global variables, lots of opportunities for mistakes
* Use column names as parameters to the standard_scaler, power transform function.